In [1]:
from my_methods import *
import requests
import io

In [2]:
dfE = pd.read_table('data/SGA_ExE.txt')
dfN = pd.read_table('data/SGA_NxN.txt')
dfEN = pd.read_table('data/SGA_ExN_NxE.txt')

df_global = pd.concat([dfE, dfN, dfEN])
df_global = df_global.dropna()
df_global = df_global.loc[df_global['P-value']<0.05]
df_global.reset_index(inplace=True)

In [3]:
qM, aM = df_to_dictionary(df_global, model='mul')
GLOBAL_mul = join_pcc(qM, aM)
GLOBAL_mul, *_ = normalize_dictionary(GLOBAL_mul)
GLOBAL_mul = select_interactions(GLOBAL_mul, 0.3)

In [4]:
qA, aA = df_to_dictionary(df_global, model='add')
GLOBAL_add = join_pcc(qA, aA)
GLOBAL_add, *_ = normalize_dictionary(GLOBAL_add)
GLOBAL_add = select_interactions(GLOBAL_add, 0.3)

In [5]:
# Interacciones en add que no estan en mul
diff_add = find_differences(GLOBAL_add, GLOBAL_mul)
diff_mul = find_differences(GLOBAL_mul, GLOBAL_add)

In [6]:
count = 0
coincidences = []
exceptions = []
filt_tables = 0.7
for tup in diff_add:
    try:
        response = requests.get(f'https://string-db.org/api/tsv/interaction_partners?identifiers={tup[0]}&species=4932&limit=100')
        table = pd.read_table(io.StringIO(response.content.decode('utf-8')))
        df = table.loc[table['score']>filt_tables]
        interactors = list(df['preferredName_B'])
        if tup[1] in interactors:
            coincidences.append(tup)
        count += 1
        print('Round: ', count, f'/{len(diff_add)}', '   Coincidences: ', len(coincidences), end='\r')
    except:
        exceptions.append(tup)
# Round:  9003 /9010    Coincidences:  314

In [1]:

count = 0
coincidences = []
exceptions = []
filt_tables = 0.7
for tup in diff_mul:
    try:
        response = requests.get(f'https://string-db.org/api/tsv/interaction_partners?identifiers={tup[0]}&species=4932&limit=100')
        table = pd.read_table(io.StringIO(response.content.decode('utf-8')))
        df = table.loc[table['score']>filt_tables]
        interactors = list(df['preferredName_B'])
        if tup[1] in interactors:
            coincidences.append(tup)
        count += 1
        print('Round: ', count, f'/{len(diff_mul)}', '   Coincidences: ', len(coincidences), end='\r')
    except:
        exceptions.append(tup)
# Round:  2512 /2515    Coincidences:  251

NameError: name 'diff_mul' is not defined